In [27]:
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

import numpy as np

# 데이터 생성

In [28]:
# 캐릭터 글자 목록
char_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# 캐릭터 사전 생성
# index순서대로 부여
char_to_idx = {c: i for i, c in enumerate(char_list)}
dic_len = len(char_to_idx)

print(char_to_idx)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [29]:
# 단어 목록
# 앞의 세 글자가 주어지면 마지막 글자를 예측
# lov -> e
word_list = ['love', 'look', 'face', 'fast', 'home', 'hope',
             'good', 'gold', 'tree', 'true', 'road', 'rock']

In [30]:
# 배치 데이터 생성
def make_batch(word_list):
    
    input_batch = []
    target_batch = []

    for word in word_list:
        # 입력 단어를 인덱스로 변환
        input = [char_to_idx[c] for c in word[:-1]]
        
        # 목표 캐릭터를 인덱스로 변환
        target = char_to_idx[word[-1]]

        # 입력 인덱스를 원핫인코딩으로 변환
        # 암배당 안하고 원핫인코딩
        input_batch.append(np.eye(dic_len)[input])

        # 목표 인덱스를 원핫인코딩으로 변환
        target_batch.append(np.eye(dic_len)[target])

    return np.array(input_batch), np.array(target_batch)

In [31]:
[char_to_idx[c] for c in 'love'[:-1]] # l:11, o:14, v:21

[11, 14, 21]

In [32]:
# 단위행렬, 정방행렬 생성
# np.identity는 단위행렬
# np.eye는 대각선인 1인 대각행렬 생성
np.eye(41)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [33]:
np.eye(dic_len)[[char_to_idx[c] for c in 'love'[:-1]]]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [34]:
# 입력/목표 배치 생성
x_train, y_train = make_batch(word_list)

In [35]:
x_train[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [36]:
y_train[0]


array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [37]:
x_train.shape # 단어 12개, time_step=3, 단어 사전 길이: 26

(12, 3, 26)

# 모델 생성

In [41]:
# 세 글자만큼 타임스텝 반복
time_step = 3

def build_model():
    
    model = Sequential()
    model.add(LSTM(64, input_shape = (time_step, dic_len)))
    model.add(Dense(dic_len, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    
    return model   

In [39]:
# summary보기 위해서 따로 출력
time_step = 3
dic_len = 26
model = Sequential()
model.add(LSTM(64, input_shape=(time_step, dic_len)))
model.add(Dense(100))
model.add(Dense(dic_len, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                23296     
_________________________________________________________________
dense (Dense)                (None, 100)               6500      
_________________________________________________________________
dense_1 (Dense)              (None, 26)                2626      
Total params: 32,422
Trainable params: 32,422
Non-trainable params: 0
_________________________________________________________________


# 훈련 및 테스트

In [42]:
# 모델 생성
model = build_model()

# 훈련 시작
model.fit(x_train,
          y_train,
          epochs=100,
          batch_size=1)


Epoch 1/100
12/12 [==============================] - 0s 1ms/step - loss: 3.2511 - acc: 0.0833
Epoch 2/100
12/12 [==============================] - 0s 1ms/step - loss: 3.1699 - acc: 0.5833
Epoch 3/100
12/12 [==============================] - 0s 1ms/step - loss: 3.0953 - acc: 0.5000
Epoch 4/100
12/12 [==============================] - 0s 1ms/step - loss: 3.0036 - acc: 0.5000
Epoch 5/100
12/12 [==============================] - 0s 1ms/step - loss: 2.8798 - acc: 0.5000
Epoch 6/100
12/12 [==============================] - 0s 1ms/step - loss: 2.7083 - acc: 0.5000
Epoch 7/100
12/12 [==============================] - 0s 1ms/step - loss: 2.4683 - acc: 0.5000
Epoch 8/100
12/12 [==============================] - 0s 914us/step - loss: 2.1592 - acc: 0.5000
Epoch 9/100
12/12 [==============================] - 0s 997us/step - loss: 1.8281 - acc: 0.5000
Epoch 10/100
12/12 [==============================] - 0s 997us/step - loss: 1.5732 - acc: 0.5000
Epoch 11/100
12/12 [==============================] -

In [43]:
# 훈련셋 데이터 예측
# 26개 캐릭터의 원핫인코딩 형식
results = model.predict(x_train)
results


array([[3.73540843e-09, 5.81029580e-09, 4.90806062e-09, 3.86591215e-04,
        9.35634792e-01, 5.19342569e-09, 2.83534929e-09, 2.63734234e-09,
        3.62371533e-09, 2.42845211e-09, 6.28069043e-02, 4.44245218e-09,
        3.61509267e-09, 3.26256466e-09, 6.08125683e-09, 4.45682913e-09,
        3.86131882e-09, 2.79461965e-09, 3.21821991e-09, 1.17171428e-03,
        4.66669192e-09, 4.93597074e-09, 3.90041643e-09, 4.07089651e-09,
        2.81788148e-09, 6.17165297e-09],
       [1.27956419e-08, 2.19287184e-08, 2.27846080e-08, 3.00206547e-03,
        1.26497254e-01, 2.29877184e-08, 1.72902599e-08, 1.35921612e-08,
        2.07044302e-08, 1.17046435e-08, 8.68782520e-01, 1.98093968e-08,
        1.44463872e-08, 1.70049361e-08, 2.72553056e-08, 2.34200623e-08,
        1.56738711e-08, 1.17746168e-08, 1.76547008e-08, 1.71801948e-03,
        1.79922672e-08, 2.53601975e-08, 1.59558091e-08, 2.02602575e-08,
        1.53299258e-08, 2.81715877e-08],
       [7.84715404e-09, 1.27351223e-08, 1.18884076e-08

In [44]:
# 1축을 기준으로 최대값의 인덱스 구함
results = np.argmax(results, 1) 
results


array([ 4, 10,  4, 19,  4,  4,  3,  3,  4,  4,  3, 10], dtype=int64)

In [45]:
# 예측 결과 출력
for i, word in enumerate(word_list):
    last_char = char_list[results[i]]
    print(word[:3] + ' -> ' + word[:3] + last_char)

lov -> love
loo -> look
fac -> face
fas -> fast
hom -> home
hop -> hope
goo -> good
gol -> gold
tre -> tree
tru -> true
roa -> road
roc -> rock
